<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>
Rafa Caballero

### Ejemplo de regresión logística

#### Carga y preprocesado

Vamos a intentar predecir en el caso del conjunto del titanic quién sobrevivió y quien no a partir de sus características

In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/titanicyesno.csv"
import pandas as pd
df = pd.read_csv(url)
df

Seleccionamos las columnas que nos interesan: survived, pclass, sex,age,sibsp,parch,fare. Queda la duda de embarked, pero vamos a incluirla por si el puerto de origen influyera de alguna forma

**Ejercicio 1** Hacer que df solo tenga las columnas indicadas

In [ ]:
columnas = ["survived", "pclass", "sex", "age", "sibsp", "parch", "fare","embarked"]
df2 = df[columnas]
df2

**Ejercicio 2**  Contar el número de nulos por columna

In [ ]:
df.isnull().sum()

In [ ]:
df [ df.age.isnull() ]

Vemos que la edad (`age`) es un problema hay una cantidad relevante de valores nulos ¿eliminamos la columna, que puede tener su importancia? ¿o las filas, que supondría la pérdida de muchas filas valiosas? Aquí vamos a optar por quitar las filas con nulos, habría que probar con el otro caso

**Ejercicio 3** Eliminar las filas con nulos

In [ ]:
# df2 = df2.drop(columns=["age"],inplace=True)
df2 = df2.drop(columns=["age"])
df2 = df2.dropna()
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
df2.dtypes

Vemos 3 columnas que no son números, integer ni float.

**Ejercicio 4** Comprobar qué  valores diferentes tienen survived, sex, embarked y cuánto se repite cada uno

In [ ]:
print(df2.survived.value_counts())
print(df2.sex.value_counts())
print(df2.embarked.value_counts())

Vemos que survived y sex son binarios; podemos utilizar `replace` para reemplazar 'yes' por 1 y 'no' por 0

In [ ]:
df2["survived"] = df2.survived.replace(('yes', 'no'), (1, 0))

O con filtros

In [ ]:
filtro  = df.survived == "yes"
df.loc[filtro,"survived"] = 1
filtro  = df.survived == "no"
df.loc[filtro,"survived"] = 0


**Ejercicio 5**
Hacer lo mismo con sex, que puede tomar 'male' o 'female'


In [ ]:
df2["sex"] = df2.sex.replace(('female', 'male'), (1, 0))

In [ ]:
df2

queda la columna `embarked`,  que, para beneficio de los algoritmos de ML queremos convertir en 3 columnas. Pandas incluye una función get_dummies que hace esto exactamente

In [ ]:
df2 = pd.get_dummies(df2, columns=["embarked"])
df2

**Nota**. Este método, llamado one-hot-encoding *no se puede aplicar a la columna de salida*. La razón es evidente: necesitamos una sola columna, no 3 por ejemplo. En este caso además el algoritmo no se equivocará por cambiar cada estring por un número ya que la etiqueta no se usa es solo el valor de salida. En este caso podemos utilizar el [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [ ]:
df2.dtypes

 Ahora estamos listos para aplicar machine learning. De los 4 pasos que describimos

1 Obtener las columnas

2 Dividir entre train y test

3 Aplicar el método

4 Evaluar

Solo cambia con respecto a la regresión lineal (y esto va a ser la tónica con el resto del aprendizaje supervisado) el 3, ya que ahora el método es diferente.

Copiar el código desde el notebook "12regresion", con los cambios

1) Antes de empezar borrar todas las filas con algún nulos,dejando el resultado en df2

2) Aplicar todos los pasos a df2

3) Las X son todas menos "survived", la y "survived"

4) El método que se aplica en el paso 3 es LogisticRegression

5) La evaluación la hacemos con la función show_results

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer,f1_score, accuracy_score, recall_score,cohen_kappa_score

def show_results(y_real,y_pred, y_pred_proba):
    print("Accuracy ",round(accuracy_score(y_real,y_pred),4),\
              "\nRecall ",round(recall_score(y_real,y_pred),4),\
              "\nKappa ", round(cohen_kappa_score(y_real,y_pred),4),
              "\nAUC ",round(roc_auc_score(y_real,y_pred_proba[:, 1]),4) )
    print('\nClassification Report: \n', classification_report(y_real, y_pred))

    RocCurveDisplay.from_predictions(y_real, y_pred)
    plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
df2 = df2.dropna()
# 1
XColumns = [c for c in df2.columns if c!="survived"]
yColumn = "survived"
X = df2[XColumns]
y = df2[yColumn]



# 2
from sklearn.model_selection import train_test_split
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3
metodo = LogisticRegression(max_iter=1000)
modelo = metodo.fit(X_train,y_train)

# 4
y_pred = modelo.predict(X_test)
y_pred_proba = modelo.predict_proba(X_test)
show_results(y_test,y_pred,y_pred_proba)

In [ ]:
print(X.columns)
modelo.coef_

In [ ]:
from  sklearn.model_selection import cross_val_score
metodo = LogisticRegression(max_iter=1000)
res = cross_val_score(metodo, X, y, cv=10, scoring="roc_auc")
res.mean()

Veamos las predicciones

In [ ]:
y_test.values,y_pred

Si lo que queremos es predecir con probabilidades

In [ ]:
y_prob = modelo.predict_proba(X_test)
y_prob[:15], y_test.values[:15], y_pred[:15]

Las probabilidades nos dan la posibilidad de alterar el etiquetado. Por ejemplo supongamos que queremos aumentar el recall de la clase 1 haciendo que se considere 1 cuando la probabilidad de ser uno sea >0.45 y no >0.5:

In [ ]:
y_pred2 = y_pred.copy()
y_pred2[y_prob[:,1]>0]=1

show_results(y_test,y_pred2,y_pred_proba)

En todo caso recordar que para obtener buenos resultados debemos ser cuidadosos y repetir el experimento varias veces

In [ ]:
from sklearn.linear_model import LogisticRegression
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler,ClusterCentroids
#from imblearn.over_sampling import SMOTE, ADASYN
from tqdm import tqdm
from sklearn.model_selection import train_test_split

df2 = df2.dropna()
# 1
XColumns = [c for c in df2.columns if c!="survived"]
yColumn = "survived"
X = df2[XColumns]
y = df2[yColumn]


# 2
veces = 500
kappa = 0
for v in tqdm(range(veces)):
    test = 0.4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)
    #X_train,y_train =ClusterCentroids().fit_resample(X_train, y_train)

    # 3
    metodo = LogisticRegression(max_iter=1000)
    modelo = metodo.fit(X_train,y_train)

    # 4
    y_pred = modelo.predict(X_test)
    kappa +=cohen_kappa_score(y_test,y_pred)

print(kappa/veces)

# 0.574